In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import medfilt

In [ ]:
%run 03_player_tracking.ipynb

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = cap.read()
assert ret

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
edges = cv2.Canny(blur, 60, 180)


lines = cv2.HoughLinesP(
    edges, 1, np.pi/180,
    threshold=120,
    minLineLength=200,
    maxLineGap=10
)

horiz = []
for [[x1,y1,x2,y2]] in lines:
    if abs(y2 - y1) < 10 and abs(x2 - x1) > 200:
        horiz.append((x1,y1,x2,y2))


In [ ]:
from collections import defaultdict

groups = defaultdict(list)

for x1,y1,x2,y2 in horiz:
    y_key = int(round(y1 / 10) * 10)
    groups[y_key].extend([x1, x2])

merged_lines = [
    (min(xs), y, max(xs), y)
    for y, xs in groups.items()
]


In [ ]:
merged_lines = sorted(merged_lines, key=lambda l: l[1])

top_baseline    = merged_lines[0]
bottom_baseline = merged_lines[-1]

h, _, _ = frame.shape
#net_line = min(merged_lines, key=lambda l: abs(l[1] - h/2))


In [ ]:
vis = frame.copy()

for x1,y1,x2,y2 in merged_lines:
    cv2.line(vis, (x1,y1), (x2,y2), (255,0,0), 2)

cv2.line(vis, top_baseline[:2], top_baseline[2:], (0,255,0), 4)
cv2.line(vis, bottom_baseline[:2], bottom_baseline[2:], (0,255,0), 4)
cv2.line(vis, net_line[:2], net_line[2:], (0,0,255), 4)

cv2.imshow("Court lines", vis)
cv2.waitKey(0)


In [ ]:
vert = []

for [[x1, y1, x2, y2]] in lines:
    if abs(x2 - x1) < 10 and abs(y2 - y1) > 200:
        vert.append((x1, y1, x2, y2))


In [ ]:
from collections import defaultdict

groups_v = defaultdict(list)

for x1, y1, x2, y2 in vert:
    x_key = int(round(x1 / 10) * 10)
    groups_v[x_key].extend([y1, y2])

merged_vert_lines = [
    (x, min(ys), x, max(ys))
    for x, ys in groups_v.items()
]

print("Vertical lines detected:", len(merged_vert_lines))
merged_vert_lines



In [ ]:
h, w = frame.shape[:2]

if len(merged_vert_lines) >= 2:
    merged_vert_lines = sorted(merged_vert_lines, key=lambda l: l[0])
    left_sideline  = merged_vert_lines[0][0]
    right_sideline = merged_vert_lines[-1][0]
else:
    # fallback to frame edges
    left_sideline  = 0
    right_sideline = w - 1


In [ ]:
vis = frame.copy()

cv2.line(vis, (int(left_sideline), 0),
               (int(left_sideline), vis.shape[0]), (0,255,0), 3)

cv2.line(vis, (int(right_sideline), 0),
               (int(right_sideline), vis.shape[0]), (0,255,0), 3)

cv2.imshow("Fallback sidelines", vis)
cv2.waitKey(0)


In [ ]:
img_pts = np.array([
    [left_sideline,  bottom_baseline[1]],
    [right_sideline, bottom_baseline[1]],
    [right_sideline, top_baseline[1]],
    [left_sideline,  top_baseline[1]],
], dtype=np.float32)


In [ ]:
for x, y in img_pts:
    cv2.circle(vis, (int(x), int(y)), 8, (0,0,255), -1)
cv2.imshow("Court detection", vis)
cv2.waitKey(0)

In [ ]:
#Real court points
world_pts = np.array([
    [0.0, 13.40],       # near-left baseline
    [5.18, 13.40],      # near-right baseline
    [5.18, 0.0],    # far-right baseline
    [0.0, 0.0],     # far-left baseline
], dtype=np.float32)

H, status = cv2.findHomography(img_pts, world_pts)

In [ ]:
traj_A = np.array(trajectories["A"])
traj_B = np.array(trajectories["B"])

def displacement(traj):
    return np.linalg.norm(np.diff(traj, axis=0), axis=1)

disp_A_raw = displacement(traj_A)
disp_B_raw = displacement(traj_B)

MAX_PX_PER_FRAME = 25

bad_A = disp_A_raw > MAX_PX_PER_FRAME
bad_B = disp_B_raw > MAX_PX_PER_FRAME

traj_A_fixed = traj_A.astype(float).copy()
traj_B_fixed = traj_B.astype(float).copy()

traj_A_fixed[1:][bad_A] = np.nan
traj_B_fixed[1:][bad_B] = np.nan


In [ ]:
def smooth_segments(traj, k=5):
    traj_s = traj.copy()
    for dim in [0, 1]:
        x = traj[:, dim]
        isnan = np.isnan(x)
        valid_idx = np.where(~isnan)[0]

        if len(valid_idx) >= k:
            x_f = x.copy()
            x_f[valid_idx] = medfilt(x[valid_idx], k)
            traj_s[:, dim] = x_f
    return traj_s

traj_A_smooth = smooth_segments(traj_A_fixed, 5)
traj_B_smooth = smooth_segments(traj_B_fixed, 5)


In [ ]:
def apply_homography(traj, H):
    """
    traj: Nx2 array (pixels)
    returns: Nx2 array (meters)
    """
    traj_m = np.full_like(traj, np.nan, dtype=float)

    valid = ~np.isnan(traj[:,0])
    pts = traj[valid].astype(np.float32).reshape(-1, 1, 2)

    mapped = cv2.perspectiveTransform(pts, H)
    traj_m[valid] = mapped.reshape(-1, 2)

    return traj_m

traj_A_m = apply_homography(traj_A_smooth, H)
traj_B_m = apply_homography(traj_B_smooth, H)

In [ ]:
def displacement_meters(traj_m):
    """
    traj_m: (N,2) in meters
    returns: (N-1,) displacement in meters
    """
    disp = np.linalg.norm(np.diff(traj_m, axis=0), axis=1)

    valid = (
        ~np.isnan(traj_m[:-1,0]) &
        ~np.isnan(traj_m[1:,0])
    )

    disp[~valid] = np.nan
    return disp


In [ ]:
disp_A_m = displacement_meters(traj_A_m)
disp_B_m = displacement_meters(traj_B_m)

# Convert to speed first (needed for temporal logic)
speed_A_ms = disp_A_m * fps
speed_B_ms = disp_B_m * fps

# Physical limits
MAX_SPEED_MPS = 9.0        # impossible beyond this
SUSPICIOUS_SPEED = 7.5     # needs temporal support

def reject_spikes(speed):
    speed_f = speed.copy()

    for i in range(1, len(speed)-1):
        if np.isnan(speed[i]):
            continue

        # Hard physical bound
        if speed[i] > MAX_SPEED_MPS:
            speed_f[i] = np.nan
            continue

        # Suspicious zone → require temporal support
        if speed[i] > SUSPICIOUS_SPEED:
            neighbors = speed[i-1:i+2]
            support = np.nansum(neighbors > SUSPICIOUS_SPEED)

            # isolated spike → reject
            if support < 2:
                speed_f[i] = np.nan

    return speed_f

speed_A_ms = reject_spikes(speed_A_ms)
speed_B_ms = reject_spikes(speed_B_ms)

# Back-compute displacement if needed later
disp_A_m = speed_A_ms / fps
disp_B_m = speed_B_ms / fps


In [ ]:
print("Player A avg speed (m/s):", np.nanmean(speed_A_ms))
print("Player B avg speed (m/s):", np.nanmean(speed_B_ms))

print("Player A peak speed (m/s):", np.nanmax(speed_A_ms))
print("Player B peak speed (m/s):", np.nanmax(speed_B_ms))


In [ ]:
def compute_acceleration(speed, fps):
    acc = np.full_like(speed, np.nan)

    for i in range(1, len(speed)):
        if np.isnan(speed[i]) or np.isnan(speed[i-1]):
            continue
        acc[i] = (speed[i] - speed[i-1]) * fps

    return acc

acc_A = compute_acceleration(speed_A_ms, fps)
acc_B = compute_acceleration(speed_B_ms, fps)

MAX_ACC = 10.0

acc_A[np.abs(acc_A) > MAX_ACC] = np.nan
acc_B[np.abs(acc_B) > MAX_ACC] = np.nan



In [ ]:
explosive_A = acc_A > 3.0
explosive_B = acc_B > 3.0

metrics = {
    "A_peak_acc": np.nanmax(acc_A),
    "B_peak_acc": np.nanmax(acc_B),
    "A_explosive_events": np.nansum(explosive_A),
    "B_explosive_events": np.nansum(explosive_B),
    "A_mean_acc": np.nanmean(acc_A > 0.0),
    "B_mean_acc": np.nanmean(acc_B > 0.0),
}


In [ ]:
metrics

In [ ]:
court_w, court_h = 5.18, 13.40
grid_size = 0.25
dt = 1.0 / fps
MIN_MOVE_SPEED = 0.3

valid_A = speed_A_ms > MIN_MOVE_SPEED
valid_B = speed_B_ms > MIN_MOVE_SPEED

pts_A = traj_A_m[1:][valid_A]
pts_B = traj_B_m[1:][valid_B]

NET_Y = 6.70
pts_A = pts_A[pts_A[:, 1] <= NET_Y]
pts_B = pts_B[pts_B[:, 1] >= NET_Y]

x_bins = np.arange(0, court_w + grid_size, grid_size)
y_bins = np.arange(0, court_h + grid_size, grid_size)

heat_A, _, _ = np.histogram2d(pts_A[:,0], pts_A[:,1], bins=[x_bins, y_bins])
heat_B, _, _ = np.histogram2d(pts_B[:,0], pts_B[:,1], bins=[x_bins, y_bins])

heat_A *= dt
heat_B *= dt


In [ ]:
plt.figure(figsize=(5, 10))

plt.imshow(
    heat_B.T,
    origin="lower",
    extent=[0, court_w, 0, court_h],
    cmap="cividis",
    interpolation="nearest"
)

plt.colorbar(label="Time spent (s)")

# NET (true physical location)
#plt.axhline(net_y, color="cyan", lw=2, label="Net")

plt.title("Player B – Full Court Coverage")
plt.xlabel("Court width (m)")
plt.ylabel("Court length (m)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(5, 10))

plt.imshow(
    heat_A.T,
    origin="lower",
    extent=[0, court_w, 0, court_h],
    cmap="cividis",
    interpolation="nearest"
)

plt.colorbar(label="Time spent (s)")

# NET (true physical location)
#plt.axhline(net_y, color="cyan", lw=2, label="Net")

plt.title("Player A – Full Court Coverage")
plt.xlabel("Court width (m)")
plt.ylabel("Court length (m)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
print("Player A y-range:", np.nanmin(traj_A_m[:,1]), np.nanmax(traj_A_m[:,1]))
print("Player B y-range:", np.nanmin(traj_B_m[:,1]), np.nanmax(traj_B_m[:,1]))

In [ ]:
print("img_pts:", img_pts)
print("traj y min/max:", np.nanmin(traj_A_smooth[:,1]), np.nanmax(traj_A_smooth[:,1]))
print("traj y min/max:", np.nanmin(traj_B_smooth[:,1]), np.nanmax(traj_B_smooth[:,1]))

In [ ]:
print("img_pts:", img_pts)
print("traj y min/max:", np.nanmin(traj_A_fixed[:,1]), np.nanmax(traj_A_fixed[:,1]))
print("traj y min/max:", np.nanmin(traj_B_fixed[:,1]), np.nanmax(traj_B_fixed[:,1]))